In [1]:
import pandas as pd
import numpy as np
import os
from collections import defaultdict

In [2]:
# table_data = pd.read_csv("/data1/yejb/prosit/figure3/supply_origin/fine-tuned.csv")
table_data = pd.read_csv(
    "/data1/yejb/prosit/figure3/supply_origin/fine-tuned-IAA-noIAA-0.01.csv")


In [3]:
# table_data[table_data['SM_v2'].apply(lambda x: bool(x))]['SM_v2']
# table_data[table_data['Sequence'].apply(lambda x: len(x) == 7)]


In [4]:

def read_allele_len(data, allele, length):
    alleles = data[data['Allele'] == allele]
    alleles = alleles[alleles['Sequence'].apply(lambda x: len(x) == length)]
    baseline = alleles[alleles['SM_v2'].apply(
        lambda x: bool(x))]['Sequence'].to_list()
    prosit = alleles[alleles['Prosit'].apply(
        lambda x: bool(x))]['Sequence'].to_list()
    finetuned = alleles[alleles['Fine-tuned Prosit'].apply(
        lambda x: bool(x))]['Sequence'].to_list()
    return baseline, prosit, finetuned

def LSG(p1, p2):
    p1 = set(p1)
    p2 = set(p2)
    return p1-p2, p1.intersection(p2), p2-p1

In [5]:
def all_unique_char(data):
    all_peptides = data["Sequence"]
    all_peptides = "".join(all_peptides)
    return set(all_peptides)

def extract_pos(peps, pos):
    return [ [p[i] for i in pos]for p in peps]

class Labeler:
    def __init__(self, unique_char):
        self._reverse = list(unique_char)
        self._dict = {
            c : i
            for i, c in enumerate(self._reverse)
        }
    def encode(self, peptide):
        return [self._dict[c] for c in peptide]

    def decode(self, pep_index):
        return "".join([self._reverse[i] for i in pep_index])
    
    def encode_batch(self, peptides):
        return [self.encode(p) for p in peptides]
    
    def decode_batch(self, pep_indexs):
        return [self.decode(p) for p in pep_indexs]
    
    def position_matrix(self, pep_indexs, length):
        mat = np.zeros((length, len(self._reverse)))    
        for p in pep_indexs:
            for i in range(length):
                mat[i, p[i]] += 1
        return mat
    
    def emission_mat(self, pep_indexs, length):
        mat = self.position_matrix(pep_indexs, length)
        return mat/(mat.sum(1).reshape(-1, 1) + 1e-9)
    
    def get_top_pos(self, pep_indexs, length, topk=5):
        mat = self.position_matrix(pep_indexs, length)
        pos_rank = np.max(mat, axis=1)
        return np.argsort(pos_rank)[::-1][:topk]

In [6]:
all_alleles = table_data['Allele'].unique()
all_length = [8, 9, 10, 11]

topk_pos = 5
labeller = Labeler(all_unique_char(table_data))


In [7]:
def emission_prob(em_mat, pep_index):
    assert len(em_mat) == len(pep_index)
    prob = 0
    for i in range(len(em_mat)):
        prob += np.log10(1e-7 + em_mat[i, pep_index[i]])
    return prob

from tqdm import tqdm

prosit_dict = defaultdict(list)
finetuned_dict = defaultdict(list)

all_length = [8, 9, 10, 11]
for allele in tqdm(all_alleles):
    for length in all_length:
        baseline, prosit, finetuned = read_allele_len(
            table_data, allele, length)
        if len(baseline) == 0:
            continue
        baseline_i = labeller.encode_batch(set(baseline + prosit + finetuned))
        needed_pos = labeller.get_top_pos(baseline_i, length, topk=topk_pos)

        inter = extract_pos(baseline_i, needed_pos)
        em_mat = labeller.emission_mat(inter, topk_pos)
        # prosit
        loss, shared, gain = LSG(baseline, prosit)
        for name, pep in zip(['Loss', 'Shared', "Gain"], [loss, shared, gain]):
            ems = []
            pep_i = labeller.encode_batch(pep)
            if len(pep_i) == 0:
                continue
            pep_i = np.array(extract_pos(pep_i, needed_pos))
            for p in pep_i:
                s = emission_prob(em_mat, p)
                ems.append(s)
            prosit_dict[name].extend(ems)
        # finetuned
        loss, shared, gain = LSG(baseline, finetuned)
        for name, pep in zip(['Loss', 'Shared', "Gain"], [loss, shared, gain]):
            ems = []
            pep_i = labeller.encode_batch(pep)
            if len(pep_i) == 0:
                continue
            pep_i = np.array(extract_pos(pep_i, needed_pos))
            for p in pep_i:
                s = emission_prob(em_mat, p)
                ems.append(s)
            finetuned_dict[name].extend(ems)


100%|██████████| 89/89 [00:54<00:00,  1.65it/s]


In [8]:
for k, v in prosit_dict.items():
    print(k, f"Mean: {np.mean(v):.3f}", f"Len: {len(v):3d}")


Loss Mean: -4.735 Len: 17490
Shared Mean: -4.284 Len: 159302
Gain Mean: -4.790 Len: 78742


In [9]:
for k, v in finetuned_dict.items():
    print(k, f"Mean: {np.mean(v):.3f}", f"Std: {len(v):.3f}")


Loss Mean: -4.945 Std: 13649.000
Shared Mean: -4.277 Std: 163143.000
Gain Mean: -6.062 Std: 144740.000


In [ ]:
from tqdm import tqdm

prosit_dict = defaultdict(list)
finetuned_dict = defaultdict(list)

all_length = [8]
for allele in tqdm(all_alleles):
    for length in all_length:
        baseline, prosit, finetuned = read_allele_len(
            table_data, allele, length)

        baseline_i = labeller.encode_batch(baseline)
        needed_pos = labeller.get_top_pos(baseline_i, length, topk=topk_pos)

        inter = extract_pos(baseline_i, needed_pos)
        baseline_train = np.array(inter)

        hmm_length = [topk_pos] * len(baseline_train)
        baseline_train = np.concatenate(baseline_train).reshape(-1, 1)
        hmm_model = hmm.MultinomialHMM(n_components=3, n_iter=50)
        hmm_model.n_features = len(labeller._reverse)
        remodel = hmm_model.fit(baseline_train, hmm_length)

        # prosit
        loss, shared, gain = LSG(baseline, prosit)
        for name, pep in zip(['Loss', 'Shared', "Gain"], [loss, shared, gain]):
            ems = []
            pep_i = labeller.encode_batch(pep)
            pep_i = np.array(extract_pos(pep_i, needed_pos))
            pep_length = [topk_pos] * len(pep_i)
            if len(pep_i) == 0:
                continue
            # pep_i = np.concatenate(pep_i).reshape(-1, 1)
            for p in pep_i:
                s = remodel.score(p.reshape(-1, 1))
                if np.isinf(s):
                    continue
                ems.append(s)
            prosit_dict[name].append(np.mean(ems))
        # finetuned
        loss, shared, gain = LSG(baseline, finetuned)
        for name, pep in zip(['Loss', 'Shared', "Gain"], [loss, shared, gain]):
            ems = []
            pep_i = labeller.encode_batch(pep)
            pep_i = np.array(extract_pos(pep_i, needed_pos))
            if len(pep_i) == 0:
                continue
            for p in pep_i:
                s = remodel.score(p.reshape(-1, 1))
                if np.isinf(s):
                    continue
                ems.append(s)
            finetuned_dict[name].append(np.mean(ems))
        # prosit_i = labeller.encode_batch(prosit)
        # finetuned_i = labeller.encode_batch(finetuned)
